# Сверточные нейронные сети


## Материал прошлой лекции

- функции активации

### Полносвязная нейронная сеть 
FCN Fully-connected Neural Net. 

В современных статьях чаще используют сокращение MLP (Multi Layer Perceptron)





На прошлом занятии мы рассотрели следующий подход:

- превращаем исходные данные в вектор 

В случае с цветным изображением 32x32 пикселя (32x32x3) этот Размер входного вектора будет равен 3072x1.

-  Перемножим(скалярно) этот вектор с матрицей весов.Размер матрицы весов может быть например 100x3072. Где 3072 - размер входа, а 100 количество признаков которые мы хотим получить. Результат будет иметь размер 100x1


- Поэлементно применяем к результату функцию обеспечивающую нелинейность, например Sigmoid или Relu (функция активации). Размерность данных при этом не меняется (100x1). Получили вектор активаций, или признаков.

- Используем признаки как входы для нового слоя. Количество весов слоя будет зависеть от размерности входа и того что мы хотим получить на выходе. Если мы делаем классификатор на 10 классов, то матрица весов должна иметь размерность 10x100 и на этом можно остановиться. Но в общем случае количество слоев может быть произвольным.


<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06_1.png" width="700">



Что нам дало добавление 2-го слоя. Мы позволили модели использовать более одного шаблона на класс. Можно убедиться в этом визуализировав веса первого слоя обученной модели.


<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/gan/mlp-templates.png" width="400">

И такая схема работает лучше чем один слой. Однако она не решает всех проблемм.
Сосредоточимся на изображениях.




## Нарушение связей между соседними пикселями

Что произойдет когда мы растягиваем изображение в вектор?

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06_9.png" width="700">

В результате такого преобразования теряется связь между соседними пикселями. 

А интуитивно понятно, что при обработке изображений важен принцип локальности. 

Набор темных пикселей например мог например составлять контур уха кошки. Но это можно понять уведетб только пока они находились на изображении рядом.

Нам необходимо найти способ как передать нейросети эту информацию. 

Идею для этого можно позаимствовать у лучшей в известном нам мире интеллектуальной системе - мозгу.

## Рецептивные поля

При анализе зрительных сбразов, наш мозг учитывает локальную информацию.

Расположенные рядом на сетчатке рецепторные клетки (палочки и колбочки) связанны с одной нервной (биполярной клеткой).  

На следующих уровнях картина повторяется (одна ганглиозная клетка связанна с несколькими более простыми биполярными) и.т.д. 

Таким образом одна более сложная клетка обрабатывает сигнал с нескольких простых рецепторов расположенных рядом и возбуждается при определенной комбинации сигналов от них.

Область сетчатки которая контролируется нервной клеткой  называется ее рецептивным полем.

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06_10.png" width="700">

На начальных уровнях этой иерарахии клетки реагируют на простые паттерны освещенности.

Например "свет в центре рецептивного поля, и отсутствие света по краям."

Далее импульсы от зрительных клеток на сетчатке попадают в талмус (LGN), а затем в зртельную кору  (V1 .. V4)


<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06_11.png" width="700">

В каждом последующем слое рецептивные поля клеток увеличиваются.
И стимулы демонстрация которыйх вызавает активацию нейронов становятся более сложными. 

##### $\color{brown}{\text{Допольнительная информация}}$

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06_12.png" width="700">

###### Hubel & Wiesel,1959

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06_13.png" width="700">

###### Ice Cube Model

https://docplayer.ru/79903347-Kasyanov-evgeniy-dmitrievich-sankt-peterburg-nmic-pn-im-v-m-beh-k-m-n-fedotov-ilya-andreevich-ryazan-ryazgmu.html

Эта гипотетическая кубическая модель придумана для пояснения устройства клеток первичной визуальной коры, а именно  – как устроены предпочитаемые ориентации и, соответственно, реакции нейронов V1. Так, V1 можно условно поделить на кубы $2 мм^3$, каждый из которых получает сигналы от обоих глаз. Клетки с одинаковыми ориентационными предпочтениями формируют горизонтальные колонки, при этом соседние вертикальные колонки имеют слегка отличающиеся ориентационные предпочтения.

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06_14.png" width="700">

Чувствительные к цветам клетки также собраны в столбцы (также их называют каплями, гиперколонками, шариками) 0,5 мм в диаметре в зонах соответствующих превалирующих глаз (картинка с цилиндрами). Каждый такой столбец содержит реагирующие либо на красно-зеленый, либо на сине-желтый контрасты.

# Свертка с фильтром


Как реализовать идею на компьютере?

Для этого испольльзуются так называемые фильтры.

Допустим нам нужно найти на изображении некий объект (например рука).

И у нас есть изображение этого объекта.

Тогда мы можем накладывать прямоугольник с изображением интересующего нас объекта на изображение, во всех возможных положениях. В какой-то момент шаблон полностью совпадет с частью рисунка.


Как определить что произошло совпадение?

В простейшем варианте можно было бы просто посчитать попиксельную разницу, и сложить результат как это делалось в KNN.

Тогда 0- соответствовал бы максимальному отклику.

Однако на практике используют умножение. Благодаря этому, чем больше соответствует фрагмент изображения шаблону тем больше будет отклик.

Это операция называется **"свертка с фильтром"**

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06-2.gif" width="300">

*В демонстрации, зеленая секция это входное изображение 5x5. Желтая матрица это фильтр, или ядро фильтра 3x3.*








In [ ]:
 # placeholder dor demo

## Примеры 'hancrafted' фильтров

Сами коэффициенты фильтра можно подбирать вручную. 
Многие известные фильтры подобранны таким образом. 

И использовать их можно не только для поиска объектов.
 

### Фильтр Гаусса

Применяется для размытия изображений, он есть в большинстве фоторедакторов.

In [ ]:
import numpy as np
import scipy.misc
from skimage import data
import matplotlib.pyplot as plt
cat = data.chelsea().mean(axis = 2).astype('int32')

def im_show(arr):
  plt.figure()
  plt.imshow(arr, cmap='gray', vmin=0, vmax=255)

im_show(cat)
import torch
g_kernel5 = torch.tensor([
                     [1,4,7,4,1],
                     [4,16,26,16,4],
                     [7,26,41,26,7],
                     [4,16,26,16,4],
                     [1,4,7,4,1]]
                     ,dtype=torch.float) / 273 # sum of weights

gaussian_filter = nn.Conv2d(in_channels=1, out_channels=1, kernel_size=g_kernel5.shape, bias=False)
gaussian_filter.weight.data = g_kernel5.unsqueeze(0).unsqueeze(0) #gaussian_kernel
gaussian_filter.weight.requires_grad = False
#input = torch.tensor(cat).unsqueeze(0).unsqueeze(0).type(torch.float)
input = torch.tensor(cat).view(1,1,cat.shape[0],cat.shape[1]).type(torch.float)
out = gaussian_filter(input)
#print(out)
im_show(out[0][0].numpy())

### Фильтр Собеля

На заре компьютерного зрения люди придумывали различные операции, которые будут находить важные элементы на изображении. К примеру, границы. Примером подобной операции является фильтр Собеля. Он у Вас будет в практической работе.

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/sobel_example.png" width="300">

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/sobel.png" width="850">

In [ ]:
# Placeholder

##### $\color{brown}{\text{Допольнительная информация}}$

###### Детектор границ Canny

На самом деле, если мы будем применять данный алгоритм без подготовки изображения и дополнительных операций, то не получим хорошее качество детекции границ. Поэтому существуют детекторы границ, которые дополняют оператор Собеля (в алгоритме ниже оператор Собеля используется на шаге 2).

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/canny.png" width="850">

###### Детекторы углов

Другими примерами подобных вручную созданных детекторов признаков являются всевозможные детекторы углов. Одним из самых известных среди них является детектор углов Харриса. Мы не будем на нем останавливаться, но вы может о нем прочесть самостоятельно.

https://en.wikipedia.org/wiki/Harris_Corner_Detector


Как Вы увидите далее, сверточные нейронные сети, по факту автоматизируют процесс нахождения подобных признаков, высвобождая время исследователей и инженеров

Перемещая различные ядра свертки по изображению, можно выполнять манипуляции с изображениями. Вот как работают многие инструменты, которые можно найти в программах для редактирования изображений, таких как Photoshop или Gimp:

###### Viola Jones ...

# Сверточный слой нейросети

Мы хотим использовать фильтры в нашей модели. При хотим что бы сеть сама научилась подбирать коэффициенты ядер фильтров. 

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06_9.png" width="700">

Вместо вектора весов размером с целое изображение используем квадратную матрицу небольшого размера.

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/gan/filter1.png" width="700">

В результате получим число, которое будет характеризовать отклик на данный фильтр. 

Операция свертки является сводится к скалярному произведению, поэтому является линейной. 

#### Цветное/многоканальное изображение

Если речь идет о цветном изображении, то входной тензор будет иметь измерение соответствующее числу цветовых каналов.

СxHxW 

Для RGB изображений число каналов равно 3.

CIFAR-10 изображения храняться в тензорах размером 3x32x32

Тогда и ядро фильтра должно получить дополнительное измерение соответствующее числу каналов. 

Например матрица весов для фильтра 3x3 будет иметь размерность 3x3x3 


<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/conv1.png" width="500">

Результатом свертки входного тензора с одним фильтром фильтром будет карта активации с глубиной 1, в независимости от количества каналов.

*При этом входной тензор не обязательно должен формироаться из пикселей изображения, а может быть выходом предидущего сверточнойго слоя. 

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06_15-1.png" width="200">


Во время прямого прохода мы смещаем фильтр по ширине и высоте входного представления и вычисляем скалярное произведение между значениями фильтра и подлежащими значениями входного представления в любой точке.

В процессе перемещения фильтра формируеется 2х мерная карта активации, которая содержит результат применения данного фильтра к каждой из областей входного представления.

В базовом случае фильтр смещается с шагов 1 но это - параметр который может меняться.




<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06_15.png" width="700">

Так же можно заметить что даже при использовании единичного шага ширина $W_{out}$ и высота $H_{out}$ **карты активаций** будет отличаться от **пространственных размерностей** исходного тензора.

In [ ]:
import numpy as np

# let's do it manually

A = np.arange(9).reshape(3,3) # our matrix
C = np.array([[-1, 0], [0, 1]]) # filter

res = [[np.sum(A[:2, :2] * C), np.sum(A[:2, 1:3] * C)],
       [np.sum(A[1:3, :2] * C), np.sum(A[1:3, 1:3] * C)]]

print("Input",A)
print("Filter",C)
print("conv",res)

Фильтров может быть несколько.

В результате обучения каждый должен обучиться реагировать на определенный паттерн во входном тензоре

Применения каждый даст свою карту активации размерностью $1xH_{out}xW_{out}$

Если объединить (стековать) эти карты между собой то получим тензор размерности $KxH_{out}xW_{out}$


Где K - количество фильтров.


<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/depth.jpeg" width="400">


На изображении результат приминения к изображению из CIFAR-10 сверточного слоя содержащего 5 фильтров.

В документации Pytorch вместо K используется обозначение $С_{out}$ 

То есть результат применения фильтра трактуется аналогично цветовому каналу на самом первом слое.



Оценим количество праметров и количество операций ...

### Сверточный слой = Сверка + активация

В результате прохода по изображению мы получаем карту активации, также известную как карта признаков. Этот процесс называется пространственной свёрткой

К изображению можно применять множество фильтров и получать на выходе разные карты активации.


По тем же соображениям что и для линейного слоя, на для каждого фильтра обучается еще обин коэффициент отвечающий за смежение (bias)


При этом сама операция свертки остается линейной операцией, и к выходу сверточного слоя применяется функция активации (например ReLu).

*Так как функция активации применяется поэлементно, не важно какой именно слой находился передней ней полносвязанный или сверточный. 


In [ ]:
import torch
from torch import nn

net = torch.nn.Sequential(
                           nn.Conv2d(3,6,3),
                           nn.ReLU(), # Activation don't depend on input shape
                           nn.Flatten(),
                           nn.Linear(5400,100), # H = 1 + (32 - 3) / 1 
                           nn.ReLU(),  # Activation don't depend on input shape
                           nn.Linear(100,10)
                           )
dummy_input = torch.randn((1,3,32,32))
out = net(dummy_input)
print(out)

## Сравнение сверточного и полносвязанного слоев 

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/gan/conv_layer.png" width="700">

- Сколько весов у такого слоя?
- А сколько операций умножения потребуется сделать при прямом проходе?

А теперь посчитаем эти же парметры для полносвязанной сети - сети?

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06_1.png" width="700">


#### Память и быстродействия

**Fully connected net**
3 слоя
FC 3x32x32 = 3072 -> 512 
512->128
128->10

W = 3072 * 512 (1 572 864) + 512* 128 (65536) + 128* 10 (1280)  = 1639680 ~= 6.25 Mb (10% от датасета CIFAR10 !)
FLOPS = 1.64M


**Convolution net**
4 слоя

3x32x32 x 10x3x3F -> 6x32x32 x 128x3x3F-> 128x32x32 -> 512x3x3F

W = (3x3x3)*10+10 = 300 
+
(3x3x3)*128 +128 = 3584
+
(3x3x3)*512 +512 = 14366
= 18220 ~= 71Kb

FLOPS = 32*32*3*3*3*10 = 276 480
+
32*32*3*3*3*128 = 3 538 944
+
32*32*3*3*512 = 14 155 776

= 17 971 200 ~= 18M


Свертки содержат значительно меньшее количество обучаемых параметров. Но требуют больше вычислительных ресурсов (GPU)

#### Fully connected layer

FC 3x32x32 = 3072 -> 512 
512->128
128->10

W = 3072 * 512 (1 572 864) + 512* 128 (65536) + 128* 10 (1280)  = 1639680 ~= 6.25 Mb (10% от датасета!)
FLOPS = 1.64M

## Параметры сверточного слоя

Основные параметры операции свертки слоя это:

- Размер ядра фильтра (F)
- Шаг (stride)

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06_18.png" width="200">

Размерность выходных данных напрямую зависит от этих параметров следующего слоя.

Для тензоров у которых высота равна ширине ( H == W == N)

Вичислить пространственне размеры выхода можно по формуле:

**$N_{new} = 1 + (( N - F ) / stride)$**

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06_19.png" width="250">


При некоторых комбинациях этих двух параметров часть данных может не попасть в свертку. 

**N = 7, F = 3**

stride 1 => (7 - 3)/1 + 1 = 5 - ОК

stride 2 => (7 - 3)/2 + 1 = 3 - ОК

stride 3 => (7 - 3)/3 + 1 = **2.33** В данном случае часть изображения в свертку не попадет

In [ ]:
import torch

# Create torch tensor 7x7
input = torch.tensor([[[[1,1,1,1,1,1,99],
                     [1,1,1,1,1,1,99],
                     [1,1,1,1,1,1,99],
                     [1,1,1,1,1,1,99],
                     [1,1,1,1,1,1,99],
                     [1,1,1,1,1,1,99]]]],dtype=torch.float)
print(input)
conv = torch.nn.Conv2d(in_channels=1, # Number of channels
                       out_channels=1, # Number of filters
                       kernel_size=3, 
                       stride = 3,
                       bias = False # Don't use bias
                       )
print(conv.weight.shape) # shape = [1,1,3,3] batch, channels, H, W
conv.weight  = torch.nn.Parameter(torch.ones((1,1,3,3))) # Replace randow weights to ones
activation = conv(input) 
print(activation)

### Количество фильтров

Задача свертки научиться разпознавать паттерны которые будут часто встречаться на изображениях в нашем датасете.

Поэтому в слой свертки добавляют не один а несколько фильтров.

От количества фильтров будет зависеть количество каналов в выходном тензоре.

Поэтому в классе Pytorch он на зывается out_channels


In [ ]:
conv = torch.nn.Conv2d(in_channels=3, # Number of input channels (3 for RGB images)
                       out_channels=64, # Number of filters
                       kernel_size=3)

dummy_input = torch.randn((1,3,32,32))
out = conv(dummy_input)
print(out.shape) # [1, 64, 30, 30] batch, C_out, H_out, W_out



### Изменение размера за счет краев

Как можно видеть из примера, при каждой операции свертки, пространственые размеры выхода уменьшаются.


В результате этого пиксели, которые находились на границе изображения участвуют в меньшем количестве сверток, чем те что были ближе к центру. 



<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06_20.png" width="900">

### Расширение (padding)  

Что бы избавиться от этого эффекта используется дополнение входного тензора (англ. padding). Обозначается как P.

 Выходы с предыдущего слоя дополняются так, чтобы после свертки размер выходного тензора совпадал с входным.

 Такие свертки называют одинаковыми (англ. same convolution), а свертки без дополнения изображения называются правильными (англ. valid convolution).

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06_21.png" width="250">

Что ба добится эффекта сохранения размера, при единичном шаге (stride == 1)

padding (P) можно рассчитать по формуле 

P = (F-1)/2


*F - размер фильтра*

 Среди способов, которыми можно заполнить новые пиксели, можно выделить следующие:

zero shift: 00[ABC]00;

border extension: AA[ABC]CC;

mirror shift: BA[ABC]CB;

cyclic shift: BC[ABC]AB.

In [ ]:
print(input.shape,input)
conv = torch.nn.Conv2d(in_channels=1, out_channels=1, kernel_size=3, stride = 1,padding = int((3-1)/2),padding_mode='zeros', bias = False)
conv.weight  = torch.nn.Parameter(torch.ones((1,1,3,3))) # Replace randow weights to ones
activation = conv(input) 
print(activation.shape,activation)

### Список параметров

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/gan/conv_sum.png" width="700">

## Визуализация

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06-3.gif" width="700">



## Операция свертки в PyTorch

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06_23.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/conv2_2.png" width="700">
<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/conv2_3.png" width="700">

# Применение сверточных слоев

### Количество слоев и размер фильтров

* Сколько сверточных слоев следует добавлять в сеть?
* Какой размер ядра фильтров использовать?

Если проводить аналогию я работой мозга, то фильтры на первых слоях сети должны реагировать на простые объекты занимающую малуч часть изображения. Или иметь малое рецептивное поле.


На последних же слоях фильтры должны реагировать на признаки описывающие целые объекты, возможно занимающие все большую часть исходного изображения.

Сответственно рецептивное поле фильтров на  последних слоях сети должно быть большим, сопоставимым с размером исходного изображения.


<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/gan/r-field.png" width="700">

Увеличить рецептивное поле можно двумя способами:

* увеличивать размер ядра фильтра
* Увеличивать количество слоев (см. схему)


Второй случай требует большого количества слоев.
И что бы уменьшить вычислительную сложность размерности входов   внутренних слоев последовательно уменьшают.



### Уменьшение размеров слоев 

- <font color=red >Не использовать расширение (padding)</font> 
- <font color=black >Увеличить шаг</font> 
- <font color=green >Свертка с фильтром 1x1</font> 
- <font color=green >Уплотнение (Субдискретизация, Pooling)</font> 

#### Уплотнение (Субдискретизация, Pooling)

Задача такая же как при сжатии обычых изображений: 

уменьшить объем данных, потеряв как можно меньше значимой информаци.

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06_24_1.png" width="700">


И способы используются аналогичные:

**Max - pooling**
Входной тензор делится на сегменты (например 2x2) в каждом канале каждого сегмента, выбирается максимальное значение.
Из этих значение составляется выходной тензор.

**Average pooling**
Аналогично за тем исключением что вместо максимального используется среднее.

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06_25.png" width="700">




Можно не делить изображение на сегменты, а а применять к нему pooling - фильтр, c шагом больше 1-цы.

Именно так операция рализуется в коде.



In [ ]:
import torch
input = torch.tensor([[[
                     [1,1,2,4],
                     [5,6,7,8],
                     [3,2,1,0],
                     [1,2,3,4],
                     ]]],dtype=torch.float)

max_pool = torch.nn.MaxPool2d(2, stride=2)
avg_pool = torch.nn.AvgPool2d(2, stride=2)

print("Max pooling",max_pool(input))
print("Average pooling",avg_pool(input))

Сответственно у слоя пулинга появляеются гиперпараметры:

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/gan/pool_sum.png" width="700">

#### Свертка с фильтром 1x1

Операция пулинга не влияет на количество каналов. А в его уплотнении тоже возникает потребность. Для этого применяют свертку размером 1x1.  


Технически это обычный сверточный слой у которого размер ядра фильтра равен 1 и шаг равен 1.

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06_24.png" width="700">

### Другие виды сверток

###1D

Свертка - это математическая операция, которая объединяет две функции в третью. Например, допустим, что у нас есть две данные функции, **е(т)** а также **г(т)** и мы заинтересованы в применении одного поверх другого и вычислении площади пересечения:**f (t) * g (t) = (f * g) (t)**

В этом случае мы применяем **г(т)**(называется ядром) более **е(т)** и изменение ответа **(Е* г) (т)** согласно пересечению над областью обеих функций. Эта концепция свертки является наиболее используемой техникой обработки сигналов, поэтому она также применяется в компьютерном зрении, что можно рассматривать как обработку сигнала от нескольких датчиков RGB. Интуиция в эффективности свертки заключается в их способности фильтровать заданные входные сигналы в объединенный, более полезный результат.

В конкретном случае изображений сигнал лучше понимается с точки зрения матриц, а не форм волны. Поэтому наши функциие **(т)а** также **г(т)** теперь станетизображение *(матрица)* а также Ядро *(матрица)* соответственно. 
Еще раз, мы можем применить свертку, сдвинув одну из матриц поверх другой:

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06.gif" width="700">

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06-1.png" width="300">

То, что вы видите выше, это просто матричное умножение скользящего окна на изображении с ядром, а затем сложение суммы. Сила сверток в контексте Computer Vision заключается в том, что они являются отличными экстракторами функций для области датчиков RGB. При индивидуальном рассмотрении каждый пиксель (датчик RGB) бессмысленно понимать, что содержит изображение. Именно отношения друг с другом в пространстве придают образу истинный смысл. Это относится к тому, как вы читаете эту статью на своем компьютере, когда пиксели, представляющие символы, и ваш мозг сопоставляют черные пиксели друг с другом в пространстве, чтобы сформировать концепцию символов.

https://pytorch.org/docs/stable/generated/torch.nn.Conv1d.html

In [ ]:
import torch
from torch import nn
#torch.nn.Conv1d(in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros')

m = nn.Conv1d(16, 33, 3, stride=2)
input = torch.randn(20, 16, 50)
output = m(input)


### 3D

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/gan/3d_conv.png" width="700">



In [ ]:
#torch.nn.Conv3d(in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros')

# With square kernels and equal stride
m = nn.Conv3d(16, 33, 3, stride=2)
# non-square kernels and unequal stride and with padding
m = nn.Conv3d(16, 33, (3, 5, 2), stride=(2, 1, 1), padding=(4, 2, 0))
input = torch.randn(20, 16, 10, 50, 100)
output = m(input)
print(output.shape)

##### $\color{brown}{\text{Допольнительная информация}}$

###### Реализация

Операция свертки может быть реализованна как скалярное произведение.

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/gan/cm1.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/gan/cm2.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/gan/cm3.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/gan/cm4.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/gan/cm5.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/gan/cm6.png" width="700">






https://medium.com/@_init_/an-illustrated-explanation-of-performing-2d-convolutions-using-matrix-multiplications-1e8de8cd2544

###### Справочник по сверткам

https://arxiv.org/pdf/1603.07285v1.pdf

https://towardsdatascience.com/types-of-convolutions-in-deep-learning-717013397f4d


the best: 
https://towardsdatascience.com/a-comprehensive-introduction-to-different-types-of-convolutions-in-deep-learning-669281e58215


## Сверточная сеть

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/gan/cnn.png" width="900">

#### LeNet

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/gan/lenet5.png" width="700">

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 3x3 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 6 * 6, 120)  # 6*6 from image dimension
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = torch.flatten(x,1) 
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()

In [ ]:
print(net)

It is a simple feed-forward network. It takes the input, feeds it through several layers one after the other, and then finally gives the output.

A typical training procedure for a neural network is as follows:

* Define the neural network that has some learnable parameters (or weights)
* Iterate over a dataset of inputs
* Process input through the network
* Compute the loss (how far is the output from being correct)
* Propagate gradients back into the network’s parameters
* Update the weights of the network, typically using a simple update rule: weight = weight - learning_rate * gradient

https://pytorch.org/tutorials/beginner/blitz/neural_networks_tutorial.html

Let’s try a random 32x32 input. Note: expected input size of this net (LeNet) is 32x32. To use this net on the MNIST dataset, please resize the images from the dataset to 32x32.

In [ ]:
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

Код для обучения не отличается от FC сети

In [ ]:
import torch.optim as optim

# create your optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)

criterion = nn.CrossEntropyLoss().cuda()

target = torch.tensor([1])  # a dummy target, for example
# in your training loop:
optimizer.zero_grad()   # zero the gradient buffers
output = net(input)

loss = criterion(output, target)
loss.backward() # backprop
optimizer.step()    # Does the update

# Визуализация сетей

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06-5.png" width="700">

In [ ]:
from torchvision import models,utils
import matplotlib.pyplot as plt
import numpy as np

plt.rcParams["figure.figsize"] = (25,25)

alexnet = models.alexnet(pretrained=True)
print(alexnet)
weight_tensor = alexnet.features[0].weight.data
print(weight_tensor.shape)

img_grid = utils.make_grid(weight_tensor,pad_value=10)
plt.imshow(np.transpose(img_grid , (1, 2, 0)))



<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06-6.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06-7.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06-8.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06-9.png" width="700">

### Visualizing Live Convnet Activations

Построение значений активации нейронов в каждом слое сети в ответ на изображение или видео. В полностью связанных нейронных сетях порядок единиц не имеет значения, именно поэтому графики этих векторов не являются пространственно информативными. Однако в сверточных сетях фильтры применяются таким образом, чтобы учитывать лежащую в основе геометрию входных данных. 

В случае же двумерных изображений фильтры применяются в ДВУМЕРНОЙ свертке по двум пространственным измерениям изображения. Эта свертка производит активации на последующих слоях, которые для каждого канала также расположены пространственно.

http://yosinski.com/media/papers/Yosinski__2015__ICML_DL__Understanding_Neural_Networks_Through_Deep_Visualization__.pdf

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06-12.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06-13.png" width="700">

### Активация слоев

Наиболее очевидный метод визуализации заключается в том, чтобы показать активации сети во время прямого прохода. Для сетей ReLU активации обычно начинает выглядеть относительно сгущенной и плотной, но по мере развития обучения активации обычно становится более редкой и локализованной. 

Единственная опасная ловушка, которую можно легко заметить с помощью этой визуализации, заключается в том, что некоторые карты активации могут быть равны нулю для многих различных входов. Это может указывать на мертвые фильтры и может быть симптомом высокой скорости обучения.

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06-14.png" width="700">

#### Окклюзирующие части изображения
Предположим, что CNN относит изображение к классу "собака". Как мы можем быть уверены, что он действительно улавливает собаку на изображении, а не контекстуальные сигналы с фона или другого объекта? Один из способов исследовать, из какой части изображения исходит некоторое классификационное предсказание, заключается в построении графика вероятности интересующего класса (например, класса собак) в зависимости от положения объекта окклюдера. 

То есть мы перебираем области изображения, устанавливаем патч изображения равным нулю и смотрим на вероятность класса. Мы можем визуализировать вероятность как 2-мерную тепловую карту. Этот подход был использован в визуализации и понимании сверточных сетей Мэтью Зейлера.

https://cs231n.github.io/understanding-cnn/

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06-15.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06-16.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06-17.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06-18.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06-19.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06-20.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06-21.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06-22.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06-23.png" width="700">

## Визуализация Модели класса

Далее разберем методику визуализации моделей классов, изученную с помощью сверточных нейронных сетей. Этот метод визуализации состоит в численном генерировании изображения, которое является репрезентативным для класса CNN. (which is representative of the class in terms of the ConvNet class scoring model.) 

Пускай Sc(I) - оценка класса c, вычисленная классификационным слоем CNN для изображения I. Мы хотели бы найти регуляризованное изображение L2, таким, чтобы показатель Sc высоким:

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/1-1.png" width="500">

где λ - параметр регуляризации. Локально-оптимальный показатель I может быть найден методом обратного распространения. Этот процесс напрямую связан с обучением CNN, где обратное распространение используется для оптимизации весов по слоям. Разница в том, что в нашем случае оптимизация выполняется по отношению к входному изображению, а веса фиксируются к тем, которые были найдены на этапе обучения.
Мы инициализировали оптимизацию с нулевым изображением (в нашем случае CNN была обучена на данных изображения с нулевым центром), а затем добавили к результату среднее изображение обучающего набора.

Стоит также отметить, что мы использовали (ненормализованные) оценки классов Sc, а не апостериоры классов, возвращаемые слоем soft-max:

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/1-2.png" width="300">

Причина в том, что максимизация класса posterior может быть достигнута за счет минимизации баллов других классов. Таким образом, мы оптимизируем Sc для того, чтобы убедиться в том, что оптимизация концентрируется только на рассматриваемом классе Sc. 

https://arxiv.org/pdf/1312.6034.pdf

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06-24.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06-25.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06-26.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06-27.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06-28.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06-29.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06-30.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06-31.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06-32.png" width="700">

## Практика

#### Pytorch. Tensor

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06_28.png" width="700">

https://pytorch.org/tutorials/beginner/blitz/neural_networks_tutorial.html